In [2]:
# Importing libraries
import numpy as np
import pandas as pd
import cv2
import os
import keras
from models import mini_xception
from keras.preprocessing.image import ImageDataGenerator

In [3]:
# Model Directory
mod_dir = "/".join(os.getcwd().split("/")[0:-1] + ['model/'])
# Data Directory
data_dir = "/".join(os.getcwd().split("/")[0:-1] + ['data/'])

In [19]:
num_classes = 7

In [20]:
def data_load():
    with open(data_dir+"fer2013.csv") as f:
        content = f.readlines()
    lines = np.array(content)
    num_of_instances = lines.size
    print("number of instances: ",num_of_instances)
    print("instance length: ",len(lines[1].split(",")[1].split(" ")))
    x_train, y_train, x_test, y_test = [], [], [], []
    for i in range(1,num_of_instances):

        emotion, img, usage = lines[i].split(",")

        val = img.split(" ")

        pixels = np.array(val, 'float32')

        emotion = keras.utils.to_categorical(emotion, num_classes)

        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)

    return x_train, y_train, x_test, y_test

In [7]:
def preprocessing(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

x_train, y_train, x_test, y_test = data_load()
x_train, y_train, x_test, y_test = preprocessing(x_train), y_train, preprocessing(x_test), y_test

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

number of instances:  35888
instance length:  2304
28709 train samples
3589 test samples


In [13]:
x_train[0]

array([[[-0.45098037],
        [-0.372549  ],
        [-0.35686272],
        ...,
        [-0.5921569 ],
        [-0.6627451 ],
        [-0.6784314 ]],

       [[-0.49019605],
        [-0.52156866],
        [-0.54509807],
        ...,
        [-0.56078434],
        [-0.5921569 ],
        [-0.654902  ]],

       [[-0.60784316],
        [-0.6627451 ],
        [-0.5764706 ],
        ...,
        [-0.6156863 ],
        [-0.56078434],
        [-0.6313726 ]],

       ...,

       [[-0.2862745 ],
        [-0.49019605],
        [-0.67058825],
        ...,
        [-0.4352941 ],
        [-0.56078434],
        [-0.6627451 ]],

       [[-0.3960784 ],
        [-0.35686272],
        [-0.38039213],
        ...,
        [-0.17647058],
        [-0.45098037],
        [-0.6392157 ]],

       [[-0.3960784 ],
        [-0.4352941 ],
        [-0.34117645],
        ...,
        [-0.16862744],
        [-0.14509803],
        [-0.35686272]]], dtype=float32)

In [9]:
# parameters
batch_size = 32
#num_epochs = 10000
input_shape = (48, 48, 1)
validation_split = .2
verbose = 1
num_classes = 7
batch_size = 256
epochs = 2

In [10]:
# Data Generator & batch preprocessing
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

train_generator = data_generator.flow(x_train, y_train, batch_size = batch_size)

# Model build
model = mini_xception(input_shape, num_classes)

# Model compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 46, 46, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 46, 46, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 46, 46, 8)    0           batch_normalization_1[0][0]      
_____________________________________

In [12]:
# Fitting the model
model.fit(
    x_train, y_train, batch_size = 256,
    epochs = epochs,
    steps_per_epoch=batch_size,
    validation_split=validation_split,
    verbose = 1
)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 28709 arrays: [array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]]), array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]]), array([[0.],
...

In [ ]:
import pandas as pd
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

#variables
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 25

with open(data_dir+"fer2013.csv") as f:
  content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

#------------------------------
#initialize trainset and test set
x_train, y_train, x_test, y_test = [], [], [], []

#------------------------------
#transfer train and test set data
for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, num_classes)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
          print("", end="")

#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#construct CNN structure
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))
#------------------------------
#batch process
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

#------------------------------

model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)

#model.fit_generator(x_train, y_train, epochs=epochs) #train for all trainset
model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=2) #train for randomly selected one

number of instances:  35888
instance length:  2304
28709 train samples
3589 test samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/2


In [ ]:
# Data Generator & batch preprocessing
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

train_generator = data_generator.flow(x_train, y_train, batch_size = batch_size)

# Model build
model = mini_xception(input_shape, num_classes)

# Model compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Fitting the model
model.fit(
    train_generator,
    epochs = epochs,
    steps_per_epoch=batch_size,
    validation_split=validation_split,
    verbose = 1
)